In [10]:
%%writefile MatrixM.cu
#include <stdio.h>

#define WIDTH 16  // Assuming a square matrix of WIDTH x WIDTH 1024

// CUDA kernel for matrix multiplication
__global__ void matrixMul(const float *A, const float *B, float *C, int width) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    float sum = 0.0;

    if (col < width && row < width) {
        for (int i = 0; i < width; i++) {
            sum += A[row * width + i] * B[i * width + col];
        }
        C[row * width + col] = sum;
    }
}


void printMatrix(const float *matrix, int width, const char* name) {
    printf("Matrix %s:\n", name);
    for (int i = 0; i < width; i++) {
        for (int j = 0; j < width; j++) {
            printf("%6.2f ", matrix[i * width + j]);
        }
        printf("\n");
    }
    printf("\n");
}



int main() {
    int width = WIDTH;
    size_t size = width * width * sizeof(float);
    float *h_A, *h_B, *h_C;  // host copies of A, B, C
    float *d_A, *d_B, *d_C;  // device copies of A, B, C

    // Allocate host memory
    h_A = (float *)malloc(size);
    h_B = (float *)malloc(size);
    h_C = (float *)malloc(size);

    // Initialize the host matrices
    for (int i = 0; i < width * width; ++i) {
        h_A[i] = rand()/(float)RAND_MAX;
        h_B[i] = rand()/(float)RAND_MAX;
    }


    // Print matrix A and B
    printMatrix(h_A, width, "A");
    printMatrix(h_B, width, "B");





    // Allocate device memory
    cudaMalloc(&d_A, size);
    cudaMalloc(&d_B, size);
    cudaMalloc(&d_C, size);

    // Copy host matrices to device
    cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);

    // Launch the matrix multiplication kernel
    dim3 dimBlock(16, 16);
    dim3 dimGrid((width + dimBlock.x - 1) / dimBlock.x, (width + dimBlock.y - 1) / dimBlock.y);
    matrixMul<<<dimGrid, dimBlock>>>(d_A, d_B, d_C, width);

    // Copy the result back to host
    cudaMemcpy(h_C, d_C, size, cudaMemcpyDeviceToHost);


    // Print result matrix C
    printMatrix(h_C, width, "C");



    // Cleanup
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
    free(h_A);
    free(h_B);
    free(h_C);

    return 0;
}


Overwriting MatrixM.cu


In [11]:
!nvcc MatrixM.cu -o matrixmulti

In [12]:
!./matrixmulti

Matrix A:
  0.84   0.78   0.91   0.34   0.28   0.48   0.36   0.95   0.64   0.14   0.02   0.14   0.16   0.13   1.00   0.51 
  0.61   0.64   0.49   0.29   0.53   0.40   0.28   0.81   0.07   0.53   0.19   0.89   0.06   0.46   0.24   0.90 
  0.27   0.38   0.51   0.53   0.44   0.93   0.28   0.64   0.69   0.44   0.83   0.23   0.35   0.96   0.66   0.44 
  0.40   0.68   0.48   0.95   0.15   0.64   0.62   0.79   0.45   0.19   0.56   0.17   0.10   0.50   0.98   0.68 
  0.75   0.29   0.58   0.15   0.13   0.16   0.07   0.05   0.18   0.80   0.66   0.64   0.09   0.52   0.07   0.46 
  0.57   0.05   1.00   0.89   1.00   0.87   0.00   0.59   0.16   0.91   0.36   0.58   0.69   0.53   0.30   0.58 
  0.75   0.04   0.83   0.87   0.98   0.90   0.67   0.16   0.89   0.65   0.63   0.70   0.33   0.07   0.22   0.51 
  0.28   0.72   0.47   0.94   0.34   0.43   0.34   0.83   0.68   0.48   0.71   0.62   0.41   0.67   0.35   0.61 
  0.73   0.74   0.92   0.65   0.53   0.26   0.69   0.11   0.58   0.67   0.78   0.33   